## Script for SQL Server

1. Query data som skal predikeres
2. Lage nødvendige features fra dataen
3. laste inn modellen
4. spytte ut predikert verdi





In [1]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
from datetime import timedelta
import datetime as dt
import pickle
import xgboost as xgb

In [19]:
query_str='''
select top 10 passenger_count 
      ,pickup_datetime  
      ,pickup_longitude
      ,pickup_latitude
      ,dropoff_longitude
      ,dropoff_latitude
      from [dbo].[nyctaxi_sample]
'''
servername = 'LNOR010941'
db = 'NYCTaxi_Sample'
conn_str = 'Driver=SQL Server;Server=' + servername + \
    ';Database=' + db + ';Trusted_Connection=True;'
cnxn = pyodbc.connect(conn_str)
#cursor = cnxn.cursor()

# Load query into dataframe
df=pd.read_sql(query_str,cnxn)

In [20]:
df['dropoff_latitude_n']=pd.to_numeric(df['dropoff_latitude'])
df['dropoff_longitude_n']=pd.to_numeric(df['dropoff_longitude'])
df['pickup_latitude_n']=pd.to_numeric(df['pickup_latitude'])
df['pickup_longitude_n']=pd.to_numeric(df['pickup_longitude'])
df=df.drop(columns=['dropoff_latitude','dropoff_longitude','pickup_latitude','pickup_longitude'])

In [21]:
nyc_airports={'JFK':{'min_lng':-73.8352,
     'min_lat':40.6195,
     'max_lng':-73.7401, 
     'max_lat':40.6659},
              
    'EWR':{'min_lng':-74.1925,
            'min_lat':40.6700, 
            'max_lng':-74.1531, 
            'max_lat':40.7081

        },
    'LaGuardia':{'min_lng':-73.8895, 
                  'min_lat':40.7664, 
                  'max_lng':-73.8550, 
                  'max_lat':40.7931
        
    }

}
def isAirport(latitude,longitude):
    res=0
    for airport_name in nyc_airports:
        if latitude>=nyc_airports[airport_name]['min_lat'] and latitude<=nyc_airports[airport_name]['max_lat'] and longitude>=nyc_airports[airport_name]['min_lng'] and longitude<=nyc_airports[airport_name]['max_lng']:
            res=1
    return res

In [5]:
def coordinates2distance(lat1,lat2,long1,long2):
    R=6373.0 #radius of earth in km
    p = 0.017453292519943295 # Pi/180 to convert to radians
    lat1=lat1*p
    lat2=lat2*p
    long1=long1*p
    long2=long2*p
    dlon=(long2-long1)
    dlat=(lat2-lat1)
    a=(np.sin(dlat/2)**2)+(np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2)
    c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    return R*c

In [22]:
df['distance']=coordinates2distance(df['pickup_latitude_n'].values,
                                    df['dropoff_latitude_n'].values,
                                    df['pickup_longitude_n'].values,
                                    df['dropoff_longitude_n'].values)

In [23]:
def encodeWeekend(day_of_week):
    day_dict={'Sunday':1,'Monday':0,'Tuesday':0,'Wednesday':0,'Thursday':0,'Friday':0,'Saturday':1}
    return day_dict[day_of_week]

In [24]:
lgr=(-73.8733, 40.7746)
jfk=(-73.7900, 40.6437)
ewr=(-74.1843, 40.6924)

In [25]:
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
df['pickup_day_of_week']= df['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
df['pickup_hour']= df['pickup_datetime'].apply(lambda x:x.hour)
df['pickup_month']= df['pickup_datetime'].apply(lambda x:x.month)
airportpickup=df.apply(lambda row:isAirport(row['pickup_latitude_n'],row['pickup_longitude_n']),axis=1)
airportdropoff=df.apply(lambda row:isAirport(row['dropoff_latitude_n'],row['dropoff_longitude_n']),axis=1)
df['is_airport']=airportpickup.combine(airportdropoff, lambda x1, x2: x1 if x1 > x2 else x2)
df['pickup_distance_jfk']=coordinates2distance(df['pickup_latitude_n'].values,
                                    jfk[1],
                                    df['pickup_longitude_n'].values,
                                    jfk[0])

df['dropoff_distance_jfk']=coordinates2distance(df['dropoff_latitude_n'].values,
                                    jfk[1],
                                    df['dropoff_longitude_n'].values,
                                    jfk[0])

df['pickup_distance_ewr']=coordinates2distance(df['pickup_latitude_n'].values,
                                    ewr[1],
                                    df['pickup_longitude_n'].values,
                                    ewr[0])

df['dropoff_distance_ewr']=coordinates2distance(df['dropoff_latitude_n'].values,
                                    ewr[1],
                                    df['dropoff_longitude_n'].values,
                                    ewr[0])
df['pickup_distance_lgr']=coordinates2distance(df['pickup_latitude_n'].values,
                                    ewr[1],
                                    df['pickup_longitude_n'].values,
                                    ewr[0])

df['dropoff_distance_lgr']=coordinates2distance(df['dropoff_latitude_n'].values,
                                    lgr[1],
                                    df['dropoff_longitude_n'].values,
                                    lgr[0])
df['is_weekend']= df['pickup_day_of_week'].apply(lambda x:encodeWeekend(x))

In [26]:
df=df.drop(['pickup_datetime','pickup_day_of_week'],axis=1)

In [12]:
modelname = 'XGBoost_model_rev2.sav'
xgb_model = pickle.load(open(modelname, 'rb'))

In [27]:
xgb_pred = xgb_model.predict(xgb.DMatrix(df), ntree_limit = xgb_model.best_ntree_limit)

In [28]:
xgb_pred

array([7.576071 , 5.334502 , 6.5551767, 6.147878 , 6.4100804, 7.197999 ,
       5.2467093, 7.3429537, 8.277509 , 6.9257154], dtype=float32)

In [31]:
query_fare='''
select top 10 fare_amount
      from [dbo].[nyctaxi_sample]
'''

# Load query into dataframe
df_fare=pd.read_sql(query_fare,cnxn)

In [32]:
df_fare

,fare_amount
0,5.5
1,5.5
2,5.5
3,5.5
4,5.5
5,5.5
6,5.5
7,5.5
8,5.5
9,5.5


In [33]:
df

,passenger_count,dropoff_latitude_n,dropoff_longitude_n,pickup_latitude_n,pickup_longitude_n,distance,pickup_hour,pickup_month,is_airport,pickup_distance_jfk,dropoff_distance_jfk,pickup_distance_ewr,dropoff_distance_ewr,pickup_distance_lgr,dropoff_distance_lgr,is_weekend
0,1,40.754913,-74.002075,40.748764,-73.988640,1.322624,12,11,0,20.425164,21.745211,17.645632,16.861508,17.645632,11.067460,1
1,1,40.763477,-73.977753,40.753613,-73.980888,1.128517,6,11,0,20.213616,20.691607,18.449548,19.121147,18.449548,8.885643,0
2,1,40.757526,-73.979164,40.750877,-73.986847,0.982869,12,12,0,20.437318,20.365351,17.870950,18.748058,17.870950,9.118316,1
3,1,40.736359,-73.988503,40.729530,-73.978096,1.160337,11,12,0,18.515769,19.659780,17.869532,17.216362,17.869532,10.597714,1
4,1,40.780495,-73.981415,40.769955,-73.987923,1.294193,10,10,0,21.811073,22.180383,18.665183,19.707963,18.665183,9.129984,0
5,1,40.718552,-73.988609,40.718441,-74.000748,1.023438,13,10,0,19.624690,18.707677,15.745865,16.755144,15.745865,11.544828,0
6,1,40.783722,-73.979683,40.788174,-73.971077,0.877797,17,12,0,22.165086,22.323010,20.889981,20.014144,20.889981,9.017570,1
7,1,40.706619,-74.003571,40.703903,-74.014412,0.962745,13,12,0,20.080874,19.328072,14.383587,15.322316,14.383587,13.330714,1
8,1,40.746857,-73.981377,40.751293,-73.994019,1.174000,14,11,0,20.957898,19.802434,17.326614,18.147595,17.326614,9.614198,1
9,1,40.709061,-74.008064,40.718910,-74.010429,1.113500,9,10,0,20.388627,19.779754,14.954393,14.976416,14.954393,13.495435,1
